In [2]:
import pandas as pd
import sqlite3

from google.colab import files
uploaded = files.upload()

df = pd.read_csv("/content/AB_NYC_2019.csv")

conn = sqlite3.connect(":memory:")
df.to_sql("airbnb", conn, index=False, if_exists="replace")


Saving AB_NYC_2019.csv to AB_NYC_2019 (1).csv


48895

In [3]:
q1 = """
SELECT id, name, neighbourhood_group, room_type, price
FROM airbnb
WHERE neighbourhood_group = 'Manhattan' AND room_type = 'Entire home/apt'
ORDER BY price DESC
LIMIT 5;
"""
pd.read_sql_query(q1, conn)

,id,name,neighbourhood_group,room_type,price
0,22436899,1-BR Lincoln Center,Manhattan,Entire home/apt,10000
1,4737930,Spanish Harlem Apt,Manhattan,Entire home/apt,9999
2,31340283,2br - The Heart of NYC: Manhattans Lower East ...,Manhattan,Entire home/apt,9999
3,23377410,Beautiful/Spacious 1 bed luxury flat-TriBeCa/Soho,Manhattan,Entire home/apt,8500
4,22779726,East 72nd Townhouse by (Hidden by Airbnb),Manhattan,Entire home/apt,7703


In [4]:
q2 = """
SELECT neighbourhood_group, room_type, COUNT(*) AS total_listings, AVG(price) AS avg_price
FROM airbnb
GROUP BY neighbourhood_group, room_type
ORDER BY total_listings DESC;
"""
pd.read_sql_query(q2, conn)

,neighbourhood_group,room_type,total_listings,avg_price
0,Manhattan,Entire home/apt,13199,249.239109
1,Brooklyn,Private room,10132,76.500099
2,Brooklyn,Entire home/apt,9559,178.327545
3,Manhattan,Private room,7982,116.776622
4,Queens,Private room,3372,71.762456
5,Queens,Entire home/apt,2096,147.050573
6,Bronx,Private room,652,66.788344
7,Manhattan,Shared room,480,88.977083
8,Brooklyn,Shared room,413,50.527845
9,Bronx,Entire home/apt,379,127.506596


In [5]:
q3 = """
SELECT id, name, price, neighbourhood_group
FROM airbnb
WHERE price > (
    SELECT AVG(price) FROM airbnb
)
ORDER BY price DESC
LIMIT 10;
"""
pd.read_sql_query(q3, conn)

,id,name,price,neighbourhood_group
0,7003697,Furnished room in Astoria apartment,10000,Queens
1,13894339,Luxury 1 bedroom apt. -stunning Manhattan views,10000,Brooklyn
2,22436899,1-BR Lincoln Center,10000,Manhattan
3,4737930,Spanish Harlem Apt,9999,Manhattan
4,9528920,"Quiet, Clean, Lit @ LES & Chinatown",9999,Manhattan
5,31340283,2br - The Heart of NYC: Manhattans Lower East ...,9999,Manhattan
6,23377410,Beautiful/Spacious 1 bed luxury flat-TriBeCa/Soho,8500,Manhattan
7,2953058,Film Location,8000,Brooklyn
8,22779726,East 72nd Townhouse by (Hidden by Airbnb),7703,Manhattan
9,33007610,70' Luxury MotorYacht on the Hudson,7500,Manhattan


In [6]:
conn.execute("""
CREATE VIEW IF NOT EXISTS neighbourhood_summary AS
SELECT neighbourhood, COUNT(*) AS total_listings, AVG(price) AS avg_price
FROM airbnb
GROUP BY neighbourhood;
""")

q4 = "SELECT * FROM neighbourhood_summary ORDER BY total_listings DESC LIMIT 10;"
pd.read_sql_query(q4, conn)


,neighbourhood,total_listings,avg_price
0,Williamsburg,3920,143.802806
1,Bedford-Stuyvesant,3714,107.678244
2,Harlem,2658,118.974041
3,Bushwick,2465,84.800406
4,Upper West Side,1971,210.918316
5,Hell's Kitchen,1958,204.794178
6,East Village,1853,186.083108
7,Upper East Side,1798,188.948276
8,Crown Heights,1564,112.480179
9,Midtown,1545,282.719094


In [7]:
conn.execute("CREATE INDEX IF NOT EXISTS idx_neighbourhood_group ON airbnb(neighbourhood_group);")